In [41]:
import pefile
import os
import re


import pandas as pd
import ast
from sklearn.preprocessing import MultiLabelBinarizer

from sklearn.feature_extraction.text import TfidfVectorizer

Obteniendo características de los malwares.

In [39]:
ruta_carpeta = './MALWR/MALWR/'

data = {}

for nombre_archivo in os.listdir(ruta_carpeta):
    ruta_archivo = os.path.join(ruta_carpeta, nombre_archivo)
    if os.path.isfile(ruta_archivo):
        pe = pefile.PE(ruta_archivo)

        # Información del PE Header
        print("Nombre del archivo: ", pe.FILE_HEADER)

        # Nombres de las secciones
        secciones = []
        for section in pe.sections:
            print(section.Name.decode().rstrip('\x00'))
            secciones.append(section.Name.decode().rstrip('\x00'))

        # Llamadas a funciones
        funciones = []
        for entry in pe.DIRECTORY_ENTRY_IMPORT:
            print('DLL: ', entry.dll.decode())
            for imp in entry.imports:
                print('\t', hex(imp.address), imp.name.decode() if imp.name else '')
                funciones.append({
                    'DLL': entry.dll.decode(),
                    'Función': imp.name.decode() if imp.name else '',
                    'Dirección': hex(imp.address)
                })

        # Cadenas de texto
        strings = list(re.findall(b"[A-Za-z0-9/\-:.,_$%'()[\]<> ]{3,}", pe.__data__))
        print('Cadenas de texto: ', strings)

        # Recursos
        recursos = []
        if hasattr(pe, 'DIRECTORY_ENTRY_RESOURCE'):
            for resource_type in pe.DIRECTORY_ENTRY_RESOURCE.entries:
                if resource_type.name is not None:
                    print('Tipo de recurso: ', resource_type.name)
                    recursos.append(resource_type.name.decode('utf-8', 'ignore'))
                else:
                    print('Tipo de recurso: ', pefile.RESOURCE_TYPE.get(resource_type.struct.Id))
                    recursos.append(pefile.RESOURCE_TYPE.get(resource_type.struct.Id))

        # Firmas digitales
        firmas = []
        if hasattr(pe, 'VS_VERSIONINFO'):
            if hasattr(pe, 'FileInfo'):
                for fileinfo in pe.FileInfo:
                    if hasattr(fileinfo, 'StringTable'):
                        for st in fileinfo.StringTable:
                            for entry in st.entries.items():
                                print(entry)
                                firmas.append(entry)

        # Almacenando la información en un diccionario.
        data[nombre_archivo] = {
            'Secciones': secciones,
            'Funciones': funciones,
            'Cadenas de texto': strings,
            'Recursos': recursos,
            'Firmas digitales': firmas
        }

df = pd.DataFrame.from_dict(data, orient='index')

df.insert(0, 'Nombre', df.index)

# Eliminando la primera columna
df = df.reset_index(drop=True)

df.to_csv('malwr.csv', index=False)

Exception: Unable to access file './MALWR/MALWR/1F2EB7B090018D975E6D9B40868C94CA': [Errno 22] Invalid argument: './MALWR/MALWR/1F2EB7B090018D975E6D9B40868C94CA'

Analizando el dataset

In [47]:
df = pd.read_csv('malwr.csv')

# Convertiendo las cadenas de texto a listas y diccionarios
for col in ['Secciones', 'Funciones', 'Cadenas de texto', 'Recursos', 'Firmas digitales']:
    df[col] = df[col].apply(ast.literal_eval)

# Transformar las listas en vectores de características
mlb = MultiLabelBinarizer()
for col in ['Secciones', 'Recursos']:
    df = df.join(pd.DataFrame(mlb.fit_transform(df.pop(col)),
                              columns=mlb.classes_,
                              index=df.index))

df

# Extrayendo solo los nombres de las funciones
df['Funciones'] = df['Funciones'].apply(lambda x: [func['Función'] for func in x])

# Transformando las listas de funciones en vectores de características
mlb = MultiLabelBinarizer()
df_funciones = pd.DataFrame(mlb.fit_transform(df.pop('Funciones')),
                            columns=mlb.classes_,
                            index=df.index)
df = df.join(df_funciones.add_prefix('Función_'))

# Convertiendo las listas de cadenas de texto a cadenas de texto
df['Cadenas de texto'] = df['Cadenas de texto'].apply(lambda x: ' '.join([s.decode('utf-8', 'ignore') for s in x]))

# Vectorizando las cadenas de texto
vectorizer = TfidfVectorizer(max_features=1000)
df_cadenas = pd.DataFrame(vectorizer.fit_transform(df.pop('Cadenas de texto')).toarray(),
                          columns=vectorizer.get_feature_names_out(),
                          index=df.index)

df = df.join(df_cadenas.add_prefix('Texto_'))

# Guardando el conjunto de datos
df.to_csv('malwr_transformed.csv', index=False)